The current setup allows to execute PySpark operations on CERN Hadoop and Spark clusters. This notebook illustrates the use of Spark in SWAN to analyze the monitoring data available on HDFS and plots a heatmap of loadAvg across machines in a particular service.

### Connect to the cluster

To connect to a cluster, click on the star button on the top and follow the instructions
* The star button only appears if you have selected a SPARK cluster in the configuration
* The star button is active after the notebook kernel is ready

### Import necessary spark and python stuff

In [1]:
from pyspark.sql.functions import from_unixtime, when, col
from pyspark.sql.types import *
from pyspark.sql.functions import from_json

In [2]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

### select the data
*path_on_hdfs_to_your_data*

In [3]:
df = spark.read.json("/user/rucio01/traces/traces.2018*")

Py4JJavaError: An error occurred while calling o61.json.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 329 in stage 2.0 failed 4 times, most recent failure: Lost task 329.3 in stage 2.0 (TID 8105, ithdp1112.cern.ch, executor 173): ExecutorLostFailure (executor 173 exited caused by one of the running tasks) Reason: Unable to create executor due to Address already in use: Service 'org.apache.spark.network.netty.NettyBlockTransferService' failed after 0 retries (starting from 33604)! Consider explicitly setting the appropriate port for the service 'org.apache.spark.network.netty.NettyBlockTransferService' (for example spark.ui.port for SparkUI) to an available port or increasing spark.port.maxRetries.
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1517)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1505)
	at org.ap

### check the structure

In [ ]:
df.printSchema()

### Filter for loadAvg
_https://metricmgr.cern.ch/metric/20002/_

In [ ]:
df.head()

In [ ]:
#body_schema = StructType([StructField("LoadAvg",DoubleType())])
#df_loadAvg = df.where(col("metric_id") == "20002").withColumn('body', from_json('body', body_schema))

### Create temporary table view

In [ ]:
# body_schema = spark.read.json(df_loadAvg.rdd.map(lambda row: row.body)).schema
df.createOrReplaceTempView("traces")

### Do the heavylifting in spark and collect aggregated view to panda DF

In [ ]:
df_pandas = spark.sql("SELECT eventVersion, \
                                      eventType, \
                                      count(*) \
                               FROM traces \
                               WHERE clientState = 'DONE'\
                               GROUP BY eventVersion, eventType")\
                    .toPandas()

### Visualize with seaborn

In [ ]:
# heatmap of service availability
df_pandas
#plt.figure(figsize=(10, 6))
#ax = sns.heatmap(df_loadAvg_pandas.pivot(index='submitter_host', columns='hr', values='avg'), cmap="Blues")
#ax.set_title("Heatmap of loadAvg")

In [ ]:
import pyspark
#df = spark.read.format("com.databricks.spark.avro").load("/user/rucio01/dumps/2018-04-24/rses")

In [ ]:
dir(pyspark)

In [ ]:
df_pandas2 = spark.sql("SELECT dataset, \
                               from_unixtime(traceTimeentryUnix, 'YYYY-MM-dd') AS dt, \
                               count(*) as amount \
                               FROM traces \
                               WHERE eventType = 'download' AND from_unixtime(traceTimeentryUnix, 'YYYY-MM-dd') >= date_add(current_date(), -30)\
                               GROUP BY 1, 2 \
                               HAVING count(1) < 10000 \
                               ORDER BY 3 DESC")\
                    .toPandas()

In [ ]:
df_pandas2

In [ ]:
df_pandas2.head()

In [ ]:
df_pandas3 = spark.sql("SELECT dataset, \
                               filename, \
                               procotol, \
                               from_unixtime(traceTimeentryUnix, 'YYYY-MM-dd') AS dt \
                               FROM traces \
                               WHERE eventType = 'download' AND from_unixtime(traceTimeentryUnix, 'YYYY-MM-dd') >= date_add(current_date(), -30)\
                               GROUP BY 1,2,3,4")\
                    .toPandas()

In [ ]:
df_pandas3

In [ ]:
sns.distplot(df_pandas2['amount'])

In [ ]:
df_pandas2.describe()

In [ ]:
plt.scatter(df_pandas2['dataset'], df_pandas2['amount']);

Make dates categorical and apply one-hot-encoding. 

In [ ]:
df_pandas2.dtypes

In [ ]:

df_pandas2.dt = pd.Categorical(df_pandas2.dt)
df_pandas2['code'] = df_pandas2.dt.cat.codes

df_pandas2

In [ ]:
sns.jointplot('code', 'amount',
              data=df_pandas2, kind="kde", color="g");

In [ ]:
df_pandas2.dtypes

In [ ]:
df_pandas2['new_date'] = pd.to_datetime(df_pandas2['dt'])


In [ ]:
df_pandas2.dtypes

In [ ]:
df_pandas2.head()

In [ ]:
import datetime

df_pandas2['cache'] = np.where(((df_pandas2['new_date'] > datetime.datetime.now() - pd.to_timedelta("30day"))&(df_pandas2['amount'] > 100)), 'yes', 'no') 



In [ ]:
df_pandas2.head()

In [ ]:
plt.scatter(df_pandas2['dt'], df_pandas2['amount']);